# Test Environment
Need to ensure that control steps in data is the same as control steps in training

In [4]:
import sys
from pathlib import Path
import matplotlib.pyplot as plt
%matplotlib inline
main_path = Path().resolve().parent
if str(main_path) not in sys.path:
    sys.path.append(str(main_path))

import os

os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.95"
os.environ["MUJOCO_GL"] = "glfw"
os.environ["XLA_FLAGS"] = (
    "--xla_gpu_enable_triton_softmax_fusion=true --xla_gpu_triton_gemm_any=True "
)
os.environ["PYOPENGL_PLATFORM"] = "glfw"

from absl import flags
import hydra
from omegaconf import DictConfig, OmegaConf
import uuid

from pathlib import Path

import functools
import jax
from typing import Dict
import wandb
import imageio
from brax import envs
from dm_control import mjcf as mjcf_dm
from dm_control.locomotion.walkers import rescale

import track_mjx.agent.custom_ppo as ppo
from track_mjx.agent import custom_ppo
from brax.io import model
import numpy as np
import pickle
import warnings
from jax import numpy as jp

from track_mjx.environment.task.multi_clip_tracking import MultiClipTracking
from track_mjx.environment.task.single_clip_tracking import SingleClipTracking
# from track_mjx.io.preprocess.mjx_preprocess import process_clip_to_train
# from track_mjx.io import preprocess as preprocessing  # the pickle file needs it
from track_mjx.environment import custom_wrappers
from track_mjx.agent import custom_ppo_networks
#from track_mjx.agent.logging import setup_training_logging
from track_mjx.io import load
from track_mjx.environment.walker.rodent import Rodent
from track_mjx.environment.walker.fly import Fly
from track_mjx.environment.walker.mouse_arm import MouseArm

from track_mjx.environment.task.reward import RewardConfig

FLAGS = flags.FLAGS
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [5]:
import yaml

main_dir = Path().resolve().parent
print(main_dir)
if str(main_dir) not in sys.path:
    sys.path.append(str(main_dir))
    
config_path = "track_mjx/config"
data_path = main_dir / "../stac-mjx/mouse_arm_ik_only_new.h5"
cfg = load.load_configs(main_dir / config_path, "mouse-arm.yaml")

env_args = cfg["env_config"]["env_args"]
env_rewards = cfg["env_config"]["reward_weights"]
# train_cfg = cfg["train_setup"]["train_config"]
walker_cfg = cfg["walker_config"]
# traj_cfg = cfg["reference_config"]
walker_type = cfg["walker_type"]
traj_config = cfg["reference_config"]
try:
    n_devices = jax.device_count(backend="gpu")
    print(f"Using {n_devices} GPUs")
except:
    n_devices = 1
    print("Not using GPUs")

envs.register_environment("mouse_arm_multi_clip", MultiClipTracking)
envs.register_environment("rodent_single_clip", SingleClipTracking)
envs.register_environment("rodent_multi_clip", MultiClipTracking)
envs.register_environment("fly_multi_clip", MultiClipTracking)

reference_clip = load.make_multiclip_data(data_path)

walker_map = {
    "mouse-arm": MouseArm,
    "rodent": Rodent,
    "fly": Fly,
}
walker_class = walker_map[walker_type]
walker = walker_class(**walker_cfg)

reward_config = RewardConfig(
    #too_far_dist = env_rewards["too_far_dist"],
    #bad_pose_dist = env_rewards["bad_pose_dist"],
    #bad_quat_dist = env_rewards["bad_quat_dist"],
    ctrl_cost_weight = env_rewards["ctrl_cost_weight"],
    ctrl_diff_cost_weight = env_rewards["ctrl_diff_cost_weight"],
    #pos_reward_weight = env_rewards["pos_reward_weight"],
    #quat_reward_weight = env_rewards["quat_reward_weight"],
    joint_reward_weight = env_rewards["joint_reward_weight"],
    #angvel_reward_weight = env_rewards["angvel_reward_weight"],
    bodypos_reward_weight = env_rewards["bodypos_reward_weight"],
    endeff_reward_weight = env_rewards["endeff_reward_weight"],
    #healthy_z_range = env_rewards["healthy_z_range"],
    #pos_reward_exp_scale = env_rewards["pos_reward_exp_scale"],
    #quat_reward_exp_scale = env_rewards["quat_reward_exp_scale"],
    joint_reward_exp_scale = env_rewards["joint_reward_exp_scale"],
    #angvel_reward_exp_scale = env_rewards["angvel_reward_exp_scale"],
    bodypos_reward_exp_scale = env_rewards["bodypos_reward_exp_scale"],
    endeff_reward_exp_scale = env_rewards["endeff_reward_exp_scale"],
    #penalty_pos_distance_scale = jp.array(env_rewards["penalty_pos_distance_scale"]),
)

env = envs.get_environment(
    env_name=cfg["env_config"]["env_name"],
    reference_clip=reference_clip,
    walker=walker,
    reward_config=reward_config,
    **env_args,
    **traj_config,
)

print("Environment created successfully!")

/root/vast/eric/track-mjx
Using 4 GPUs
env._steps_for_cur_frame: 2.0
Environment created successfully!


In [6]:
rollout_env = custom_wrappers.AutoResetWrapperTracking(
    custom_wrappers.RenderRolloutWrapperTracking(env)
    )
jit_reset = jax.jit(rollout_env.reset)
jit_step = jax.jit(rollout_env.step)
key = jax.random.PRNGKey(0)

In [7]:
_, key = jax.random.split(key)
state = jit_reset(key)

/root/miniforge3/envs/track_mjx_env/lib/python3.11/site-packages/jax/_src/interpreters/xla.py:132: RuntimeWarning: overflow encountered in cast
  return np.asarray(x, dtypes.canonicalize_dtype(x.dtype))


In [8]:
rollout = [state]
frames = [env._get_cur_frame(state.info, state.pipeline_state)]
for i in range(10):
    _, key = jax.random.split(key)
    action = jax.random.normal(key, shape=(env.action_size,))
    state = jit_step(state, action)
    rollout.append(state)
    frames.append(env._get_cur_frame(state.info, state.pipeline_state))

In [24]:
qpos_rollout = [s.pipeline_state.qpos for s in rollout]
qpos_rollout

[Array([-0.21430749, -0.6041761 , -0.2602248 ,  0.5402993 ], dtype=float32),
 Array([-0.2243591 , -0.61410964, -0.25004506,  0.46475792], dtype=float32),
 Array([-0.21419743, -0.6042364 , -0.2405095 ,  0.51129085], dtype=float32),
 Array([-0.22441918, -0.61410916, -0.24993965,  0.46686628], dtype=float32),
 Array([-0.21419027, -0.6241106 , -0.24050854,  0.5106302 ], dtype=float32),
 Array([-0.20416042, -0.6341084 , -0.23055437,  0.6050643 ], dtype=float32),
 Array([-0.2141029 , -0.6441087 , -0.24050257,  0.70636004], dtype=float32),
 Array([-0.22434953, -0.65410227, -0.23007725,  0.65866005], dtype=float32),
 Array([-0.23413427, -0.6442309 , -0.24045229,  0.7099329 ], dtype=float32),
 Array([-0.22448336, -0.63422906, -0.22999422,  0.6609264 ], dtype=float32),
 Array([-0.2142629 , -0.6441128 , -0.24037533,  0.7121609 ], dtype=float32)]

In [ ]:
plt.plot(np.arange(len(frames)), frames)

In [ ]:
state = env.reset(rng_key)
state.pipeline_state.qpos[:3]

In [ ]:
reference_clip = jax.tree_map(
            lambda x: x[state.info["cur_frame"]], env._get_reference_clip(state.info)
        )
reference_clip.position